In [1]:
import pandas as pd

In [6]:
# read data from data repo
raw_data_csv = pd.read_csv('https://raw.githubusercontent.com/GuyattDL/web/main/Absenteeism_data.csv')

In [7]:
raw_data_csv

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
695,17,10,23/05/2018,179,22,40,237.656,22,2,2,0,8
696,28,6,23/05/2018,225,26,28,237.656,24,1,1,2,3
697,18,10,24/05/2018,330,16,28,237.656,25,2,0,0,8
698,25,23,24/05/2018,235,16,32,237.656,25,3,0,0,2


In [9]:
# copy df to and drop ID column
df = raw_data_csv.copy()
df.drop(['ID'], axis=1, inplace=True)

In [11]:
# create dummy variables for 'Reason for Absence' and check for MECE
reason_dummies = pd.get_dummies(df['Reason for Absence'])
reason_dummies['check'] = reason_dummies.sum(axis=1)
reason_dummies['check'].unique()

array([1])

In [15]:
# all rows sum to 1 and only 1, now drop first column to avoid multicolinearity
reason_dummies = pd.get_dummies(df['Reason for Absence'], drop_first=True)
reason_dummies

,1,2,3,4,5,6,7,8,9,10,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [16]:
# Categories should be grouped to reduce number of features.
# From the documentation we can group the reasons as follows:
# Reason 1: Disease related
# Reason 2: Pregnancy related
# Reason 3: Poisoning related or not elsewhere categorised
# Reason 4: 'Light' reasons

In [18]:
reason_type_1 = reason_dummies.loc[:,1:14].max(axis=1)
reason_type_2 = reason_dummies.loc[:,15:17].max(axis=1)
reason_type_3 = reason_dummies.loc[:,18:21].max(axis=1)
reason_type_4 = reason_dummies.loc[:,22:].max(axis=1)

In [19]:
# concatenate onto df
df = pd.concat([df,reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis=1)
df.head()

,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,0,1,2,3
0,26,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,0,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,23,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,7,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [20]:
df.drop(['Reason for Absence'], axis=1)
df.columns.values

array(['Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours', 0, 1, 2, 3], dtype=object)

In [22]:
# rename and reorder columns
column_names = ['Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours', 'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']

df.columns = column_names

cols_reordered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours']

df = df[cols_reordered]
df.head()


,Reason_1,Reason_2,Reason_3,Reason_4,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [ ]:
# create repo, push work to date to Github
# next steps: date to datetime, extract and Month and Weekday...
